In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error

In [ ]:
file_path = "podaci.xlsx"
df = pd.read_excel(file_path)

In [ ]:
current_year = pd.to_datetime("today").year
df["Године"] = current_year - pd.to_datetime(df["Датум рођења"]).dt.year
df["Старосна група"] = pd.cut(df["Године"], bins=[0, 18, 30, 50, 100], labels=["<18", "18-30", "30-50", "50+"])
print(df.describe())

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корелациона матрица")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корелациона матрица")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="Старосна група", y="Висина (цм)", data=df)
plt.title("Висина у функцији старосне групе")
plt.show()

In [ ]:
anova_p = stats.f_oneway(df[df["Старосна група"] == "18-30"]["Висина (цм)"],
                         df[df["Старосна група"] == "30-50"]["Висина (цм)"],
                         df[df["Старосна група"] == "50+"]["Висина (цм)"])
print("ANOVA p-vrednost za visinu:", anova_p.pvalue)

In [ ]:
X = df[["Дужина главе", "Ширина главе", "Ширина лица"]]
y = df["Старосна група"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression()
}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} Klasifikacija:\n", classification_report(y_test, y_pred))

In [ ]:
X = df[["Године"]]
y = df["Висина (цм)"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridge = Ridge().fit(X_train, y_train)
lasso = Lasso().fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
y_pred_lasso = lasso.predict(X_test)
print("Ridge MSE:", mean_squared_error(y_test, y_pred_ridge))
print("Lasso MSE:", mean_squared_error(y_test, y_pred_lasso))

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(df[["Дужина главе", "Ширина главе", "Ширина лица"]])
df["Кластер"] = kmeans.labels_
sns.scatterplot(x=df["Ширина главе"], y=df["Ширина лица"], hue=df["Кластер"], palette="viridis")
plt.title("Кластери по облику главе")
plt.show()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.iloc[:, 3:-1])
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.title("PCA Analiza")
plt.xlabel("Glavna komponenta 1")
plt.ylabel("Glavna komponenta 2")
plt.show()